In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

# Librerías básicas
import time
import numpy as np
import pandas as pd
import joblib
import json
import pickle
import gc
import sys
import warnings
from datetime import datetime
from glob import glob
from collections import defaultdict

# Librerías de machine learning
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, roc_curve, auc
)

# Configuración
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

# Variables globales de configuración
RANDOM_STATE = 42
MEMORY_EFFICIENT = True
MAX_MODELS_IN_MEMORY = 5

# Variables globales de rutas de archivos
models_base_dir = "modelos"
results_base_dir = "results"
evaluation_results_dir = "results/evaluation_results"
logs_dir = "logs"
evaluation_data_file = "datos/evaluation_data.parquet"

# Nombres de archivos de salida
csv_output_filename = "evaluation_results.csv"
excel_output_filename = "evaluation_complete.xlsx"
json_output_filename = "evaluation_detailed.json"

# Extensiones de archivos
model_file_extension = ".joblib"
metadata_file_suffix = "_metadata.json"

# Crear directorios necesarios
os.makedirs(evaluation_results_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

In [2]:
def get_memory_usage(obj):
    """Calcula el uso de memoria de un objeto en MB"""
    if isinstance(obj, np.ndarray):
        return obj.nbytes / (1024 * 1024)
    elif isinstance(obj, list):
        total_size = sys.getsizeof(obj)
        for item in obj:
            if isinstance(item, np.ndarray):
                total_size += item.nbytes
            else:
                total_size += sys.getsizeof(item)
        return total_size / (1024 * 1024)
    else:
        return sys.getsizeof(obj) / (1024 * 1024)

def discover_models(models_dir=None):
    """Descubre modelos desde estructura fija: modelos/[Algorithm]/[Encoding].joblib"""
    if models_dir is None:
        models_dir = models_base_dir
        
    print(f"Cargando modelos desde '{models_dir}'")
    
    if not os.path.exists(models_dir):
        print(f"El directorio {models_dir} no existe")
        return []
    
    discovered_models = []
    
    # Algoritmos esperados
    algorithms = ['SVM', 'RandomForest', 'XGBoost']
    
    # Buscar en cada directorio de algoritmo
    for algorithm in algorithms:
        algorithm_dir = os.path.join(models_dir, algorithm)
        
        if not os.path.exists(algorithm_dir):
            print(f"Directorio {algorithm} no encontrado")
            continue
        
        # Buscar archivos .joblib en el directorio del algoritmo
        for file in os.listdir(algorithm_dir):
            if file.endswith(model_file_extension):
                encoding = os.path.splitext(file)[0]
                model_file = os.path.join(algorithm_dir, file)
                
                # Buscar archivo de metadata correspondiente
                metadata_file = os.path.join(algorithm_dir, f"{encoding}{metadata_file_suffix}")
                metadata = {}
                
                if os.path.exists(metadata_file):
                    try:
                        with open(metadata_file, 'r') as f:
                            metadata = json.load(f)
                    except Exception as e:
                        print(f"Error cargando metadata para {algorithm}/{file}: {e}")
                
                model_info = {
                    'algorithm': algorithm,
                    'encoding': encoding,
                    'model_file': model_file,
                    'metadata_file': metadata_file if os.path.exists(metadata_file) else None,
                    'metadata': metadata
                }
                
                discovered_models.append(model_info)
                print(f"  Encontrado: {algorithm} - {encoding}")
    
    print(f"Total de modelos descubiertos: {len(discovered_models)}")
    return discovered_models

def load_model_safely(model_path):
    """Carga un modelo de forma segura"""
    try:
        start_time = time.time()
        model = joblib.load(model_path)
        load_time = time.time() - start_time
        return model, load_time, None
    except Exception as e:
        return None, 0, str(e)

def load_evaluation_data(data_file=None):
    """Carga datos de evaluación"""
    if data_file is None:
        data_file = evaluation_data_file
    
    if os.path.exists(data_file):
        try:
            print(f"Cargando datos desde {data_file}")
            if data_file.endswith('.parquet'):
                data = pd.read_parquet(data_file)
            else:
                data = pd.read_csv(data_file)
            print(f"Datos cargados exitosamente: {len(data)} filas")
            return data
        except Exception as e:
            print(f"Error cargando {data_file}: {e}")
    
    print(f"No se pudo cargar el archivo de datos: {data_file}")
    return None

def create_class_mapping(evaluation_data):
    """Crea mapeo de clases basado en los datos"""
    if 'clases_modelos' in evaluation_data.columns and 'genus' in evaluation_data.columns:
        # Crear mapeo desde genus
        mapping_data = evaluation_data[['genus', 'clases_modelos']].drop_duplicates()
        class_mapping = dict(zip(mapping_data['clases_modelos'], mapping_data['genus']))
        print(f"Mapeo de clases creado: {len(class_mapping)} clases")
        return class_mapping, evaluation_data
    else:
        print("No se encontraron columnas de clase válidas")
        return None, evaluation_data

In [ ]:

def calculate_unified_metrics(y_true, y_pred, y_proba=None, class_mapping=None):
    """
    Función unificada para calcular métricas detalladas.
    Compatible con ambos códigos.
    """
    
    # Métricas globales - UNIFICADAS
    global_metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'f1_score_weighted': f1_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted'),
        'f1_score_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'f1_micro': f1_score(y_true, y_pred, average='micro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'n_samples': len(y_true),
        'n_classes': len(np.unique(y_true))
    }
    
    # Matriz de confusión
    cm = confusion_matrix(y_true, y_pred)
    
    # Métricas por clase
    unique_classes = np.unique(y_true)
    class_metrics = {
        'sensitivity_per_class': {},
        'specificity_per_class': {},
        'precision_per_class': {},
        'recall_per_class': {}
    }
    
    # Si no hay mapeo de clases, crear uno genérico
    if class_mapping is None:
        class_mapping = {i: f"Class_{i}" for i in unique_classes}
    
    for i, class_id in enumerate(unique_classes):
        class_name = class_mapping.get(class_id, f"Class_{class_id}")
        
        # Calcular TP, FP, FN, TN
        TP = cm[i, i] if i < cm.shape[0] and i < cm.shape[1] else 0
        FP = np.sum(cm[:, i]) - TP if i < cm.shape[1] else 0
        FN = np.sum(cm[i, :]) - TP if i < cm.shape[0] else 0
        TN = np.sum(cm) - (TP + FP + FN)
        
        # Métricas por clase
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        class_metrics['sensitivity_per_class'][class_name] = recall
        class_metrics['specificity_per_class'][class_name] = specificity
        class_metrics['precision_per_class'][class_name] = precision
        class_metrics['recall_per_class'][class_name] = recall
    
    # ROC AUC por clase si hay probabilidades
    roc_auc_scores = {}
    if y_proba is not None:
        try:
            for i, class_id in enumerate(unique_classes):
                if i < y_proba.shape[1]:
                    class_name = class_mapping.get(class_id, f"Class_{class_id}")
                    y_true_binary = (y_true == class_id).astype(int)
                    
                    if len(np.unique(y_true_binary)) > 1:
                        fpr, tpr, _ = roc_curve(y_true_binary, y_proba[:, i])
                        roc_auc_scores[class_name] = auc(fpr, tpr)
        except Exception as e:
            print(f"Error calculando ROC AUC: {e}")
    
    return {
        'global_metrics': global_metrics,
        'class_metrics': class_metrics,
        'confusion_matrix': cm.tolist(),
        'roc_auc_scores': roc_auc_scores,
        # Agregar métricas individuales para compatibilidad
        'accuracy': global_metrics['accuracy'],
        'f1_score_weighted': global_metrics['f1_score_weighted'],
        'f1_score_macro': global_metrics['f1_score_macro'],
        'precision_weighted': global_metrics['precision_weighted'],
        'precision_macro': global_metrics['precision_macro'],
        'recall_weighted': global_metrics['recall_weighted'],
        'recall_macro': global_metrics['recall_macro'],
        'f1_weighted': global_metrics['f1_weighted'],
        'f1_macro': global_metrics['f1_macro'],
        'f1_micro': global_metrics['f1_micro'],
        'n_samples': global_metrics['n_samples'],
        'n_classes': global_metrics['n_classes']
    }

def calculate_detailed_metrics(y_true, y_pred, y_proba, class_mapping):
    """Función para el primer código - mantiene la interfaz original"""
    return calculate_unified_metrics(y_true, y_pred, y_proba, class_mapping)

def evaluate_single_model(model_info, evaluation_data, class_mapping):
    """Evalúa un modelo individual - VERSIÓN SIMPLIFICADA MEJORADA"""
    
    algorithm = model_info['algorithm']
    encoding = model_info['encoding']
    model_file = model_info['model_file']
    
    print(f"Evaluando {algorithm} - {encoding}")
    
    result = {
        'algorithm': algorithm,
        'encoding': encoding,
        'model_file': model_file,
        'success': False,
        'error': None,
        'training_metadata': model_info.get('metadata', {}),
        'evaluation_metrics': {},
        'timing_info': {},
        'memory_info': {},
        'predictions': {}
    }
    
    try:
        if encoding not in evaluation_data.columns:
            raise ValueError(f"Encoding {encoding} no encontrado en datos de evaluación")
        
        # Cargar modelo
        print(f"  Cargando modelo...")
        model, load_time, load_error = load_model_safely(model_file)
        
        if model is None:
            raise ValueError(f"Error cargando modelo: {load_error}")
        
        result['timing_info']['model_load_time'] = load_time
        
        # Preparar datos - FILTRADO MENOS AGRESIVO
        print(f"  Preparando datos...")
        data_prep_start = time.time()
        
        X_eval_raw = evaluation_data[encoding].tolist()
        y_eval_raw = evaluation_data['clases_modelos'].values
        
        print(f"  Datos originales: {len(X_eval_raw)} muestras")
        
        # CORRECCIÓN: Filtrado más inteligente
        valid_indices = []
        for i, x in enumerate(X_eval_raw):
            is_valid = False
            
            if x is not None:
                if isinstance(x, str) and x.strip() and x.lower() not in ['nan', 'null', 'none']:
                    is_valid = True
                elif isinstance(x, (list, np.ndarray)) and len(x) > 0:
                    if isinstance(x, np.ndarray):
                        is_valid = not np.all(np.isnan(x))
                    else:
                        is_valid = True
                elif isinstance(x, (int, float)) and not (np.isnan(x) or np.isinf(x)):
                    is_valid = True
                else:
                    # Intentar convertir otros tipos
                    try:
                        test_array = np.array(x)
                        if test_array.size > 0 and not np.all(np.isnan(test_array)):
                            is_valid = True
                    except:
                        pass
            
            if is_valid:
                valid_indices.append(i)
        
        print(f"  Datos válidos después de filtrado: {len(valid_indices)}")
        
        if len(valid_indices) == 0:
            raise ValueError("No hay datos válidos para evaluación")
        
        X_eval_clean = [X_eval_raw[i] for i in valid_indices]
        y_eval_clean = y_eval_raw[valid_indices]
        
        # Conversión más robusta a numpy array
        try:
            X_eval_array = np.array(X_eval_clean)
            
            if X_eval_array.ndim == 1 and len(X_eval_clean) > 0:
                first_element = X_eval_clean[0]
                if hasattr(first_element, '__len__') and not isinstance(first_element, str):
                    try:
                        X_eval_array = np.vstack([np.array(x).flatten() for x in X_eval_clean])
                    except ValueError:
                        # Normalizar a la dimensión más común
                        lengths = [len(np.array(x).flatten()) for x in X_eval_clean[:min(100, len(X_eval_clean))]]
                        most_common_length = max(set(lengths), key=lengths.count)
                        
                        X_eval_processed = []
                        for x in X_eval_clean:
                            x_flat = np.array(x).flatten()
                            if len(x_flat) == most_common_length:
                                X_eval_processed.append(x_flat)
                            elif len(x_flat) > most_common_length:
                                X_eval_processed.append(x_flat[:most_common_length])
                            else:
                                padded = np.zeros(most_common_length)
                                padded[:len(x_flat)] = x_flat
                                X_eval_processed.append(padded)
                        
                        X_eval_array = np.vstack(X_eval_processed)
                        
        except Exception as e:
            raise ValueError(f"Error procesando datos del encoding {encoding}: {e}")
        
        print(f"  Shape final de datos: {X_eval_array.shape}")
        
        data_prep_time = time.time() - data_prep_start
        result['timing_info']['data_prep_time'] = data_prep_time
        
        # Información de memoria
        result['memory_info'] = {
            'input_data_mb': get_memory_usage(X_eval_array),
            'original_samples': len(X_eval_raw),
            'valid_samples': len(X_eval_clean),
            'feature_dimensions': X_eval_array.shape[1] if len(X_eval_array.shape) > 1 else 1
        }
        
        # Hacer predicciones
        print(f"  Realizando predicciones...")
        prediction_start = time.time()
        
        y_pred = model.predict(X_eval_array)
        
        # Obtener probabilidades
        y_proba = None
        try:
            if hasattr(model, 'predict_proba'):
                y_proba = model.predict_proba(X_eval_array)
                print(f"  Probabilidades obtenidas: shape {y_proba.shape}")
        except Exception as e:
            print(f"  Warning: No se pudieron obtener probabilidades: {e}")
        
        prediction_time = time.time() - prediction_start
        result['timing_info']['prediction_time'] = prediction_time
        result['timing_info']['total_time'] = load_time + data_prep_time + prediction_time
        
        # Calcular métricas
        print(f"  Calculando métricas...")
        metrics_start = time.time()
        
        detailed_metrics = calculate_unified_metrics(y_eval_clean, y_pred, y_proba, class_mapping)
        
        metrics_time = time.time() - metrics_start
        result['timing_info']['metrics_calc_time'] = metrics_time
        
        result['evaluation_metrics'] = detailed_metrics
        
        # CORRECCIÓN CLAVE: Guardar TODAS las predicciones para visualización
        result['predictions'] = {
            'y_true': y_eval_clean.tolist(),
            'y_pred': y_pred.tolist(),
            'y_proba': y_proba.tolist() if y_proba is not None else None,
            'correct_predictions': (y_eval_clean == y_pred).tolist(),
            'accuracy': detailed_metrics['accuracy'],
            'n_samples_used': len(y_eval_clean),
            'n_samples_original': len(X_eval_raw)
        }
        
        if result['timing_info']['total_time'] > 0:
            result['efficiency_score'] = detailed_metrics['accuracy'] / result['timing_info']['total_time']
        else:
            result['efficiency_score'] = 0
        
        result['success'] = True
        
        print(f"  Completado - Accuracy: {detailed_metrics['accuracy']:.4f}, "
              f"F1 Weighted: {detailed_metrics['f1_score_weighted']:.4f}, "
              f"Muestras utilizadas: {len(y_eval_clean)}/{len(X_eval_raw)}")
        
        # Liberar memoria
        if MEMORY_EFFICIENT:
            del model, X_eval_array, y_pred
            if y_proba is not None:
                del y_proba
            gc.collect()
        
    except Exception as e:
        result['error'] = str(e)
        print(f"  Error: {e}")
    
    return result

def save_evaluation_results(all_results, results_dir=None):
    """Versión mejorada del guardado - OPCIONAL"""
    
    if results_dir is None:
        results_dir = evaluation_results_dir
    
    # Crear DataFrame principal
    main_data = []
    for result in all_results:
        if result['success']:
            metrics = result['evaluation_metrics']
            timing = result['timing_info']
            memory = result['memory_info']
            predictions = result.get('predictions', {})
            
            row = {
                'algorithm': result['algorithm'],
                'encoding': result['encoding'],
                'accuracy': metrics['accuracy'],
                'f1_weighted': metrics['f1_weighted'],
                'f1_score_weighted': metrics['f1_score_weighted'],
                'f1_macro': metrics['f1_macro'],
                'f1_score_macro': metrics['f1_score_macro'],
                'f1_micro': metrics['f1_micro'],
                'precision_weighted': metrics['precision_weighted'],
                'precision_macro': metrics['precision_macro'],
                'recall_weighted': metrics['recall_weighted'], 
                'recall_macro': metrics['recall_macro'], 
                'n_samples': metrics['n_samples'],
                'n_classes': metrics['n_classes'],
                'total_time': timing['total_time'],
                'prediction_time': timing['prediction_time'],
                'model_load_time': timing['model_load_time'],
                'efficiency_score': result['efficiency_score'],
                'input_data_mb': memory['input_data_mb'],
                'feature_dimensions': memory['feature_dimensions'],
                'model_file': result['model_file'],
                # NUEVO: Información de muestras
                'valid_samples': predictions.get('n_samples_used', memory.get('valid_samples', 0)),
                'original_samples': predictions.get('n_samples_original', memory.get('original_samples', 0)),
                'has_probabilities': predictions.get('y_proba') is not None
            }
            main_data.append(row)
    
    if not main_data:
        print("No hay resultados exitosos para guardar")
        return None
    
    df_main = pd.DataFrame(main_data)
    
    # Guardar CSV principal
    csv_path = os.path.join(results_dir, csv_output_filename)
    df_main.to_csv(csv_path, index=False)
    print(f"Resultados principales guardados: {csv_path}")
    
    # Guardar Excel con múltiples hojas
    excel_path = os.path.join(results_dir, excel_output_filename)
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df_main.to_excel(writer, sheet_name='Resumen', index=False)
        
        # Métricas por clase
        class_data = []
        for result in all_results:
            if result['success'] and result['evaluation_metrics']['class_metrics']:
                for class_name in result['evaluation_metrics']['class_metrics']['precision_per_class'].keys():
                    row = {
                        'algorithm': result['algorithm'],
                        'encoding': result['encoding'],
                        'class_name': class_name,
                        'precision': result['evaluation_metrics']['class_metrics']['precision_per_class'][class_name],
                        'recall': result['evaluation_metrics']['class_metrics']['recall_per_class'][class_name],
                        'sensitivity': result['evaluation_metrics']['class_metrics']['sensitivity_per_class'][class_name],
                        'specificity': result['evaluation_metrics']['class_metrics']['specificity_per_class'][class_name]
                    }
                    class_data.append(row)
        
        if class_data:
            df_class = pd.DataFrame(class_data)
            df_class.to_excel(writer, sheet_name='Metricas_por_Clase', index=False)
        
        # Timing
        timing_data = []
        for result in all_results:
            if result['success']:
                row = {
                    'algorithm': result['algorithm'],
                    'encoding': result['encoding'],
                    **result['timing_info']
                }
                timing_data.append(row)
        
        if timing_data:
            df_timing = pd.DataFrame(timing_data)
            df_timing.to_excel(writer, sheet_name='Tiempos_Ejecucion', index=False)
    
    print(f"Excel completo guardado: {excel_path}")
    
    # Guardar JSON CON todas las predicciones
    json_path = os.path.join(results_dir, json_output_filename)
    
    json_data = {
        'total_models': len(all_results),
        'successful_evaluations': len([r for r in all_results if r['success']]),
        'failed_evaluations': len([r for r in all_results if not r['success']]),
        'results': []
    }
    
    # CLAVE: Incluir TODAS las predicciones en el JSON
    for result in all_results:
        if result['success']:
            json_result = {
                'algorithm': result['algorithm'],
                'encoding': result['encoding'],
                'model_file': result['model_file'],
                'success': result['success'],
                'accuracy': float(result['evaluation_metrics']['accuracy']),
                'f1_weighted': float(result['evaluation_metrics']['f1_weighted']),
                'f1_macro': float(result['evaluation_metrics']['f1_macro']),
                'precision_weighted': float(result['evaluation_metrics']['precision_weighted']),
                'precision_macro': float(result['evaluation_metrics']['precision_macro']),
                'recall_weighted': float(result['evaluation_metrics']['recall_weighted']),
                'recall_macro': float(result['evaluation_metrics']['recall_macro']),
                'total_time': float(result['timing_info']['total_time']),
                'efficiency_score': float(result['efficiency_score']),
                # CLAVE: Incluir todas las predicciones
                'predictions': result.get('predictions', {}),
                'evaluation_metrics': {
                    'class_metrics': result['evaluation_metrics'].get('class_metrics', {}),
                    'confusion_matrix': result['evaluation_metrics'].get('confusion_matrix', None),
                    'roc_auc_scores': result['evaluation_metrics'].get('roc_auc_scores', {})
                }
            }
            json_data['results'].append(json_result)
    
    try:
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        print(f"JSON guardado: {json_path}")
    except Exception as e:
        print(f"Error guardando JSON: {e}")
    
    return {
        'csv_path': csv_path,
        'excel_path': excel_path,
        'json_path': json_path,
        'main_dataframe': df_main
    }

In [4]:

# ============================================================================
# EJECUCIÓN PRINCIPAL
# ============================================================================

print("Buscando modelos...")
# Descubrir modelos
discovered_models = discover_models("modelos")  # Cambia la ruta según tu estructura

print("Cargando datos de evaluación...")
# Cargar datos de evaluación
evaluation_data = load_evaluation_data("datos/evaluation_data.parquet")

print("Mapeando clases...")
class_mapping, evaluation_data = create_class_mapping(evaluation_data)

if class_mapping is None:
    print("Error: No se pudo crear el mapeo de clases")
    exit(1)

print(f"Datos cargados: {len(evaluation_data)} muestras, {len(class_mapping)} clases")

Buscando modelos...
Cargando modelos desde 'modelos'
  Encontrado: SVM - PS_K-mers + FFT
  Encontrado: SVM - PS_FFT
  Encontrado: SVM - AS_K-mers + Wavelet
  Encontrado: SVM - PS_One Hot + FFT
  Encontrado: SVM - AS_One Hot + Wavelet
  Encontrado: SVM - PS_K-mers
  Encontrado: SVM - PS_One Hot
  Encontrado: SVM - AS_One Hot
  Encontrado: SVM - AS_K-mers
  Encontrado: SVM - AS_One Hot + FFT
  Encontrado: SVM - PS_K-mers + Wavelet
  Encontrado: SVM - AS_K-mers + FFT
  Encontrado: SVM - PS_Wavelet
  Encontrado: SVM - AS_FFT
  Encontrado: SVM - PS_One Hot + Wavelet
  Encontrado: SVM - AS_Wavelet
  Encontrado: RandomForest - PS_K-mers + FFT
  Encontrado: RandomForest - PS_FFT
  Encontrado: RandomForest - AS_K-mers + Wavelet
  Encontrado: RandomForest - PS_One Hot + FFT
  Encontrado: RandomForest - AS_One Hot + Wavelet
  Encontrado: RandomForest - PS_K-mers
  Encontrado: RandomForest - PS_One Hot
  Encontrado: RandomForest - AS_One Hot
  Encontrado: RandomForest - AS_K-mers
  Encontrado: Ran

In [5]:

# Evaluar modelos
print("Iniciando evaluación de modelos...")

all_results = []
successful_count = 0
failed_count = 0

total_models = len(discovered_models)

for i, model_info in enumerate(discovered_models):
    print(f"Progreso: {i+1}/{total_models}")
    
    # Verificar que el archivo existe
    if not os.path.exists(model_info['model_file']):
        print(f"Archivo no encontrado: {model_info['model_file']}")
        failed_count += 1
        continue
    
    # Evaluar modelo
    result = evaluate_single_model(model_info, evaluation_data, class_mapping)
    all_results.append(result)
    
    if result['success']:
        successful_count += 1
    else:
        failed_count += 1
    
    # Liberar memoria cada ciertos modelos
    if MEMORY_EFFICIENT and i % MAX_MODELS_IN_MEMORY == 0:
        gc.collect()

print(f"Evaluación completada: {successful_count} exitosos, {failed_count} fallidos")

Iniciando evaluación de modelos...
Progreso: 1/48
Evaluando SVM - PS_K-mers + FFT
  Cargando modelo...
  Preparando datos...
  Datos originales: 8555 muestras
  Datos válidos después de filtrado: 8555
  Shape final de datos: (8555, 798)
  Realizando predicciones...
  Probabilidades obtenidas: shape (8555, 29)
  Calculando métricas...
  Completado - Accuracy: 0.9582, F1 Weighted: 0.9578, Muestras utilizadas: 8555/8555
Progreso: 2/48
Evaluando SVM - PS_FFT
  Cargando modelo...
  Preparando datos...
  Datos originales: 8555 muestras
  Datos válidos después de filtrado: 8555
  Shape final de datos: (8555, 799)
  Realizando predicciones...
  Probabilidades obtenidas: shape (8555, 29)
  Calculando métricas...
  Completado - Accuracy: 0.9607, F1 Weighted: 0.9600, Muestras utilizadas: 8555/8555
Progreso: 3/48
Evaluando SVM - AS_K-mers + Wavelet
  Cargando modelo...
  Preparando datos...
  Datos originales: 8555 muestras
  Datos válidos después de filtrado: 8555
  Shape final de datos: (8555, 7

In [6]:
# Guardar resultados
print("Guardando resultados...")
saved_files = save_evaluation_results(all_results)

# Resumen final
if saved_files and 'main_dataframe' in saved_files:
    df = saved_files['main_dataframe']
    if not df.empty:
        print(f"\n{'='*60}")
        print(f"EVALUACIÓN COMPLETADA")
        print(f"{'='*60}")
        print(f"Modelos evaluados exitosamente: {successful_count}")
        print(f"Modelos fallidos: {failed_count}")
        print(f"Total procesados: {len(all_results)}")
        print(f"\nMEJORES RESULTADOS:")
        print(f"Mejor accuracy: {df['accuracy'].max():.4f}")
        print(f"Accuracy promedio: {df['accuracy'].mean():.4f}")
        print(f"Mejor F1 weighted: {df['f1_weighted'].max():.4f}")
        print(f"Mejor precision macro: {df['precision_macro'].max():.4f}")
        print(f"Mejor eficiencia: {df['efficiency_score'].max():.4f}")
        
        best_model = df.loc[df['accuracy'].idxmax()]
        print(f"\nMEJOR MODELO:")
        print(f"  Algoritmo: {best_model['algorithm']}")
        print(f"  Encoding: {best_model['encoding']}")
        print(f"  Accuracy: {best_model['accuracy']:.4f}")
        print(f"  F1 Weighted: {best_model['f1_weighted']:.4f}")
        print(f"  Precision Macro: {best_model['precision_macro']:.4f}")
        print(f"  Tiempo: {best_model['total_time']:.2f}s")
        
        print(f"\nARCHIVOS GENERADOS:")
        for key, path in saved_files.items():
            if key != 'main_dataframe':
                print(f"  {key}: {path}")
        print(f"{'='*60}")

Guardando resultados...
Resultados principales guardados: results/evaluation_results/evaluation_results.csv
Excel completo guardado: results/evaluation_results/evaluation_complete.xlsx
JSON guardado: results/evaluation_results/evaluation_detailed.json

EVALUACIÓN COMPLETADA
Modelos evaluados exitosamente: 48
Modelos fallidos: 0
Total procesados: 48

MEJORES RESULTADOS:
Mejor accuracy: 0.9953
Accuracy promedio: 0.7715
Mejor F1 weighted: 0.9953
Mejor precision macro: 0.9955
Mejor eficiencia: 4.7688

MEJOR MODELO:
  Algoritmo: SVM
  Encoding: AS_One Hot
  Accuracy: 0.9953
  F1 Weighted: 0.9953
  Precision Macro: 0.9955
  Tiempo: 565.57s

ARCHIVOS GENERADOS:
  csv_path: results/evaluation_results/evaluation_results.csv
  excel_path: results/evaluation_results/evaluation_complete.xlsx
  json_path: results/evaluation_results/evaluation_detailed.json
